In [8]:
import os

def  load_ai_hub_data(path):
    for root, dirs,files in os.walk(path):
        for file in files:
            audio_path=os.path.join(root,file)
            text_file=file.replace(".wav", ".txt")
            text_path=os.path.join(root.replace("원천데이터", "라벨링데이터"), text_file)

            if os.path.exists(text_path):
                try:
                    with open(text_path, "r", encoding="utf-8") as f:
                        text = f.read().strip()
                        data.append({"audio_path": audio_path, "text": text})
                except Exception as e:
                    print(f"Error reading {text_path}: {e}")
    return data

In [10]:
base_data_path = "./dummy_ai_hub_data"

# (실제 데이터셋이 없으므로, 더미 파일 생성 로직 추가)
# 이 부분은 실제 데이터셋을 다운로드했다면 건너뛰세요.
if not os.path.exists(base_data_path):
    os.makedirs(os.path.join(base_data_path, "원천데이터"), exist_ok=True)
    os.makedirs(os.path.join(base_data_path, "라벨링데이터"), exist_ok=True)
    for i in range(5): # 5개의 더미 파일 생성
        dummy_wav_path = os.path.join(base_data_path, "원천데이터", f"dummy_{i:03d}.wav")
        dummy_txt_path = os.path.join(base_data_path, "라벨링데이터", f"dummy_{i:03d}.txt")
        # 더미 .wav 파일 (실제 데이터는 아니지만 경로 존재를 위해 생성)
        with open(dummy_wav_path, "wb") as f:
            f.write(b"RIFF\x00\x00\x00\x00WAVEfmt \x10\x00\x00\x00\x01\x00\x01\x00\x80>\x00\x00\x00\x00\x00\x00\x02\x00\x10\x00data\x00\x00\x00\x00") # Minimal WAV header
        with open(dummy_txt_path, "w", encoding="utf-8") as f:
            f.write(f"안녕하세요 더미 텍스트입니다 {i}")

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchaudio
import os
import unicodedata
from transformers import AutoProcessor, AutoModelForCTC, Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor
import json
from tqdm.auto import tqdm # tqdm 임포트 확인

# --- 0. 환경 설정 (이전 코드에서 재사용) ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model_name = "facebook/wav2vec2-base-960h"

# --- 1. Custom Tokenizer Creation for Wav2Vec2 Model (재사용) ---
vocab_elements = [
    "ㄱ", "ㄴ", "ㄷ", "ㄹ", "ㅁ", "ㅂ", "ㅅ", "ㅇ", "ㅈ", "ㅊ", "ㅋ", "ㅌ", "ㅍ", "ㅎ",
    "ㄲ", "ㄸ", "ㅃ", "ㅆ", "ㅉ",
    "ㅏ", "ㅑ", "ㅓ", "ㅕ", "ㅗ", "ㅛ", "ㅜ", "ㅠ", "ㅡ", "ㅣ", "ㅐ", "ㅔ", "ㅚ", "ㅟ", "ㅢ",
    "ㅘ", "ㅝ", "ㅙ", "ㅞ",
    "0", "1", "2", "3", "4", "5", "6", "7", "8", "9",
    ",", ".", "?", "!", "-",
    " ", "|", "<unk>", "<pad>"
]
unique_vocab = sorted(list(set([v for v in vocab_elements if v not in [" ", "|", "<unk>", "<pad>"]])))
final_vocab_list = unique_vocab + [" ", "|", "<unk>", "<pad>"]

vocab_dict = {token: i for i, token in enumerate(final_vocab_list)}

with open("vocab.json", "w", encoding="utf-8") as f:
    json.dump(vocab_dict, f, ensure_ascii=False)

tokenizer = Wav2Vec2CTCTokenizer(
    "vocab.json",
    unk_token="<unk>",
    pad_token="<pad>",
    word_delimiter_token=" "
)

feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1,
    sampling_rate=16000,
    padding_value=0.0,
    do_normalize=True,
    return_attention_mask=False
)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
final_vocab_size = processor.tokenizer.vocab_size
print(f"Processor configured. Final Vocab Size: {final_vocab_size}")

# --- 2. 모델 로드 (재사용) ---
model = AutoModelForCTC.from_pretrained(
    model_name,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=final_vocab_size,
    ignore_mismatched_sizes=True
)
model.lm_head = torch.nn.Linear(model.config.hidden_size, final_vocab_size)
model.to(device)
print(f"Model '{model_name}' loaded and lm_head modified. lm_head.out_features: {model.lm_head.out_features}")

# --- 3. 데이터 콜레이터 (재사용) ---
class DataCollatorCTCWithPadding:
    def __init__(self, processor):
        self.processor = processor

    def __call__(self, features):
        audio_samples = [feature["audio"] for feature in features]
        text_labels = [feature["text"] for feature in features]

        padded_audio_batch = self.processor.feature_extractor.pad(
            {"input_values": audio_samples},
            return_tensors="pt",
            padding=True
        )
        batch_input_values = padded_audio_batch["input_values"]

        with self.processor.as_target_processor():
            tokenized_labels = []
            for label in text_labels:
                encoded_label = self.processor.tokenizer(label).input_ids
                if encoded_label:
                    tokenized_labels.append(encoded_label)
                else:
                    print(f"Warning: Empty tokenized label for text: '{label}'. Using default.")
                    default_text = "안녕"
                    decomposed_default_text = "".join(unicodedata.normalize("NFD", char) for char in default_text if unicodedata.category(char) != 'Mn')
                    tokenized_labels.append(self.processor.tokenizer(decomposed_default_text).input_ids)

            labels_batch = self.processor.tokenizer.pad(
                {"input_ids": tokenized_labels},
                padding=True,
                return_tensors="pt"
            )

        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch = {
            "input_values": batch_input_values,
            "labels": labels
        }
        return batch

# --- 4. AIHubSpeechDataset 클래스 정의 (NEW) ---
class AIHubSpeechDataset(Dataset):
    def __init__(self, data_list, processor, sample_rate=16000):
        self.data_list = data_list
        self.processor = processor
        self.sample_rate = sample_rate

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        item = self.data_list[idx]
        audio_path = item["audio_path"]
        text_label = item["text"]

        # 1. 오디오 로드 및 리샘플링
        # torchaudio.load는 (waveform, sample_rate) 튜플을 반환
        try:
            waveform, sr = torchaudio.load(audio_path)
            if sr != self.sample_rate:
                # 리샘플러를 생성하고 적용
                resampler = torchaudio.transforms.Resample(sr, self.sample_rate)
                waveform = resampler(waveform)
            
            # 스테레오 오디오인 경우 모노로 변환 (첫 번째 채널만 사용)
            if waveform.shape[0] > 1:
                waveform = waveform[0, :] # 첫 번째 채널만 선택
            else:
                waveform = waveform.squeeze(0) # (1, sequence_length) -> (sequence_length)
            
            # Wav2Vec2 모델의 feature extractor는 1D 넘파이 배열 또는 파이토치 텐서를 기대함
            audio_array = waveform.numpy() # 또는 .tolist()
            
        except Exception as e:
            # 오디오 로드 실패 시 경고 출력 및 더미 데이터 반환
            print(f"Warning: Could not load audio {audio_path}: {e}. Returning dummy data.")
            audio_array = torch.randn(self.sample_rate * 2).numpy() # 2초 길이의 더미 오디오
            text_label = "오류 발생 더미 텍스트" # 더미 텍스트도 설정

        # 2. 텍스트 전처리 (한글 자모 분리)
        # Wav2Vec2CTCTokenizer는 한글 자모 단위를 예상하므로 NFD 정규화 필요
        # 단, 한글이 아닌 문자는 정규화하지 않도록 유니코드 카테고리 'Mn'(Mark, Nonspacing) 제외
        # (이전 코드에서 텍스트 생성 시 이 부분을 이미 고려했으므로, 여기서는 한 번 더 확인차 적용)
        decomposed_text = "".join(unicodedata.normalize("NFD", char) for char in text_label if unicodedata.category(char) != 'Mn')
        
        # 라벨링 데이터 전사 시 간혹 긴 공백이 있을 수 있으므로 공백 정규화
        decomposed_text = " ".join(decomposed_text.split())
        
        return {"audio": audio_array, "text": decomposed_text}

# --- 5. 데이터셋 로드 및 DataLoader 준비 ---

# 이 부분은 실제 AI Hub 데이터셋 경로로 바꿔주세요!
# 예: base_ai_hub_path = "/path/to/your/AI_Hub_Korean_Speech_Dataset/일반인_음성/01.초등학생"
# 현재는 더미 데이터를 생성하여 사용하겠습니다.

base_data_path = "./dummy_ai_hub_data"

# (실제 데이터셋이 없다면 이 더미 파일 생성 로직을 사용)
if not os.path.exists(base_data_path):
    os.makedirs(os.path.join(base_data_path, "원천데이터"), exist_ok=True)
    os.makedirs(os.path.join(base_data_path, "라벨링데이터"), exist_ok=True)
    print("Dummy AI Hub data directories created.")
    for i in range(10): # 10개의 더미 파일 생성 (훈련 데이터셋 크기 조절)
        dummy_wav_path = os.path.join(base_data_path, "원천데이터", f"dummy_{i:03d}.wav")
        dummy_txt_path = os.path.join(base_data_path, "라벨링데이터", f"dummy_{i:03d}.txt")
        # 더미 .wav 파일 (torchaudio가 로드할 수 있는 간단한 오디오 데이터 생성)
        # 16000Hz, 1초 길이의 사인파
        sample_rate = 16000
        duration = 1 # seconds
        frequency = 440 # Hz
        t = torch.linspace(0, duration, int(sample_rate * duration), requires_grad=False)
        dummy_waveform = 0.5 * torch.sin(2 * torch.pi * frequency * t)
        torchaudio.save(dummy_wav_path, dummy_waveform.unsqueeze(0), sample_rate) # (channels, samples) 형태로 저장

        with open(dummy_txt_path, "w", encoding="utf-8") as f:
            f.write(f"테스트 문장 번호 {i} 입니다") # 실제 한글 텍스트 사용

print(f"Loading data from: {base_data_path}")

def load_ai_hub_data_paths(base_path):
    data = []
    # AI Hub 데이터셋의 실제 구조에 따라 이 로직을 수정해야 합니다!
    # 여기서는 '원천데이터'와 '라벨링데이터'가 같은 부모 아래에 있다고 가정합니다.
    # 각 음성 파일의 메타데이터 파일(JSON/TXT) 위치를 정확히 파악해야 합니다.
    
    # 예시: '원천데이터' 폴더를 탐색하며 .wav 파일과 그에 매칭되는 .txt 라벨링 파일을 찾음
    audio_data_dir = os.path.join(base_path, "원천데이터")
    label_data_dir = os.path.join(base_path, "라벨링데이터")

    if not os.path.exists(audio_data_dir) or not os.path.exists(label_data_dir):
        print(f"Warning: '{audio_data_dir}' or '{label_data_dir}' not found. Check AI Hub data structure.")
        return []

    for filename in os.listdir(audio_data_dir):
        if filename.endswith(".wav"):
            audio_path = os.path.join(audio_data_dir, filename)
            
            # AI Hub 데이터는 파일명만 같고 확장자가 다른 경우가 많음
            # 예: xxx.wav -> xxx.txt
            text_filename = filename.replace(".wav", ".txt")
            text_path = os.path.join(label_data_dir, text_filename)

            if os.path.exists(text_path):
                try:
                    with open(text_path, "r", encoding="utf-8") as f:
                        text = f.read().strip()
                        if text: # 텍스트 내용이 비어있지 않은 경우만 추가
                            data.append({"audio_path": audio_path, "text": text})
                        else:
                            print(f"Skipping empty text for {audio_path}")
                except Exception as e:
                    print(f"Error reading text for {audio_path}: {e}")
            else:
                print(f"Warning: Text file not found for {audio_path} at {text_path}. Skipping.")
    return data

# 실제 데이터 로드
ai_hub_data_list = load_ai_hub_data_paths(base_data_path)
print(f"로드된 실제 데이터 샘플 수: {len(ai_hub_data_list)}")

if not ai_hub_data_list:
    print("오류: 로드된 데이터가 없습니다. AI Hub 데이터셋 경로와 구조를 다시 확인해주세요!")
    print("훈련을 계속하려면 최소한의 데이터가 필요합니다.")
else:
    # 데이터셋 인스턴스 생성
    train_dataset = AIHubSpeechDataset(ai_hub_data_list, processor)

    # DataLoader 인스턴스 생성
    data_collator_instance = DataCollatorCTCWithPadding(processor=processor)

    train_loader = DataLoader(
        train_dataset,
        batch_size=4, # 배치 크기는 하드웨어 사양에 따라 조절
        shuffle=True,
        collate_fn=data_collator_instance,
        num_workers=0 # 멀티 프로세싱 (Windows에서는 0으로 설정하는 것이 안전)
    )
    print(f"훈련 데이터셋 크기: {len(train_dataset)} 샘플, 배치 수: {len(train_loader)}")

    # --- 6. 훈련 환경 설정 (Optimizer) 재사용 ---
    lr = 1e-4
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    print(f"Optimizer configured (AdamW, LR={lr})")

    # --- 7. 훈련 루프 (재사용) ---
    num_epochs = 3 # 예시 에포크 수

    model.train() # 모델을 훈련 모드로 설정

    print("\n--- Training Started with AI Hub-like Data ---")
    for epoch in range(num_epochs):
        total_loss = 0
        # tqdm으로 훈련 진행 상황 표시
        for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}")):
            input_val = batch["input_values"].to(device)
            label = batch["labels"].to(device)

            optimizer.zero_grad() # 그라디언트 초기화

            outputs = model(input_values=input_val, labels=label)
            loss = outputs.loss # CTC 손실 값

            loss.backward() # 역전파
            optimizer.step() # 옵티마이저 스텝

            total_loss += loss.item() # 손실 누적

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")

    print("\n--- Training Completed ---")

C:\Users\JH\anaconda\envs\ai_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda
Processor configured. Final Vocab Size: 57


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized because the shapes did not match:
- lm_head.bias: found shape torch.Size([32]) in the checkpoint and torch.Size([57]) in the model instantiated
- lm_head.weight: found shape torch.Size([32, 768]) in the checkpoint and torch.Size([57, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model 'facebook/wav2vec2-base-960h' loaded and lm_head modified. lm_head.out_features: 57
Loading data from: ./dummy_ai_hub_data
로드된 실제 데이터 샘플 수: 5
훈련 데이터셋 크기: 5 샘플, 배치 수: 2
Optimizer configured (AdamW, LR=0.0001)

--- Training Started with AI Hub-like Data ---


Epoch 1:   0%|          | 0/2 [00:00<?, ?it/s]C:\Users\JH\anaconda\envs\ai_env\lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:174: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Epoch 1:   0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: Calculated padded input size per channel: (0). Kernel size: (10). Kernel size can't be greater than actual input size